In [34]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_helper import rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
from PW_explorer.pwe_nb_helper import ASPRules
from PW_explorer.export import PWEExport
from PW_explorer.visualize import PWEVisualization

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
from copy import deepcopy
from collections import OrderedDict

import csv
import re
from astropy.time import Time

In [2]:
%load_ext PWE_NB_Extension

In [3]:
tableLayoutCustomer = ['c_custkey','c_name','c_address','c_nationkey','c_phone','c_acctbal','c_mktsement',
                       'c_comment']
tableLayoutLineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice',
                       'l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate',
                      'l_receiptdate','l_shipinstruct','l_shipmode','l_comment']
tableLayoutOrders = ['o_orderkey','o_custkey','o_orderstatus','o_totalprice','o_orderdate','o_orderpriority',
                     'o_clerk','o_shippriority','o_comment']
tableLayoutPart = ['p_partkey','p_name','p_mfgr','p_brand','p_type','p_size','p_container','p_rentalprice',
                   'p_comment']
tableLayoutPartsupp = ['ps_partkey','ps_suppkey','ps_availqty','ps_supplycost','ps_comment']
tableLayoutSupllier = ['s_suppkey','s_name','s_address','s_nationkey','s_phone','s_acctbal','s_comment']

schemas = [tableLayoutCustomer,tableLayoutLineitem,tableLayoutOrders,tableLayoutPart,tableLayoutPartsupp,tableLayoutSupllier]
schemaNames = ['customer','lineitem','orders','part','partsupp','supplier']

In [4]:
customer = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/ref_data/1/customer.tbl.1'
lineitem = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/ref_data/1/lineitem.tbl.1'
orders = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/ref_data/1/orders.tbl.1'
part = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/ref_data/1/part.tbl.1'
partsupp = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/ref_data/1/partsupp.tbl.1'
supplier = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/ref_data/1/supplier.tbl.1'

In [5]:
dataTables = [customer,lineitem,orders,part,partsupp,supplier]
dataValues = []

for db in dataTables:
    singleValues = []
    with open(db) as csvfile:
        readCsv = csv.reader(csvfile,delimiter='|')
        for row in readCsv:
            singleValues.append(row[:-1])
    dataValues.append(singleValues)

In [6]:
sqlQueries = []
sqldir = '/home/mj1e16/abduction/pwe/testData/tpch/2.18.0_rc2/dbgen/queries/'
for x in range(1,23):
    sqlfile = sqldir+str(x)+'.sql'
    with open(sqlfile,'r') as f:
        sqlQueries.append(f.read())

In [29]:
a ='1234-'
b = re.search('[a-zA-Z-]',a)
print(b)
if b == None:
    print('nonfsdib')

<_sre.SRE_Match object; span=(4, 5), match='-'>


In [106]:
def makeOriginal(dataValues,tableName,colour='black'):
    clingoStart = []
    for x in dataValues:
        adjustedValues = [y if re.search('[a-zA-Z-]',str(y)) == None else '"'+y+'"' for y in x]
        for i,y in enumerate(adjustedValues):
            if re.search('[-]',str(y)) != None:
                if len(y) == 12:
                    adjustedValues[i] = str(Time(y[1:-1], format='isot', scale='utc').mjd)
#                 except:
#                     print(len(y[1:-1]))
#                     print(y[1:-1])
#                     pass
        #adjustedDates = [Time(y, format='isot', scale='utc') if re.search('[-]',str(y)) == None and  else y for y in adjustedValues]
        # replace spaces here?
        clingoStart.append(tableName+'('+','.join(adjustedValues)+',black,"-",'+','.join(adjustedValues)+').')
    return clingoStart

In [14]:
clingoRules  = ''
for i,table in enumerate(schemas):
    clingoRules += '% schema '+schemaNames[i]+'('+','.join(table)+')\n'

In [15]:
ASPRules(clingoRules)

'% schema customer(c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsement,c_comment)\n% schema lineitem(l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment)\n% schema orders(o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment)\n% schema part(p_partkey,p_name,p_mfgr,p_brand,p_type,p_size,p_container,p_rentalprice,p_comment)\n% schema partsupp(ps_partkey,ps_suppkey,ps_availqty,ps_supplycost,ps_comment)\n% schema supplier(s_suppkey,s_name,s_address,s_nationkey,s_phone,s_acctbal,s_comment)'

In [107]:
originalSetup = []
for tab in range(len(dataValues)):
    originalSetup.extend(makeOriginal(dataValues[tab],schemaNames[tab]))

In [108]:
ASPRules('\n'.join(originalSetup))

'customer(1,"Customer#000000001","IVhzIApeRb ot,c,E",15,"25-989-741-2988",711.56,"BUILDING","to the even, regular platelets. regular, ironic epitaphs nag e",black,"-",1,"Customer#000000001","IVhzIApeRb ot,c,E",15,"25-989-741-2988",711.56,"BUILDING","to the even, regular platelets. regular, ironic epitaphs nag e").\ncustomer(2,"Customer#000000002","XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121.65,"AUTOMOBILE","l accounts. blithely ironic theodolites integrate boldly: caref",black,"-",2,"Customer#000000002","XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,"23-768-687-3665",121.65,"AUTOMOBILE","l accounts. blithely ironic theodolites integrate boldly: caref").\ncustomer(3,"Customer#000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498.12,"AUTOMOBILE"," deposits eat slyly ironic, even instructions. express foxes detect slyly. blithely even accounts abov",black,"-",3,"Customer#000000003","MG9kdTD2WBHm",1,"11-719-748-3364",7498.12,"AUTOMOBILE"," deposits eat slyly ironic, even instructions. express foxes detect slyly. blithely even accounts abov").\ncustomer(4,"Customer#000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866.83,"MACHINERY"," requests. final, regular ideas sleep final accou",black,"-",4,"Customer#000000004","XxVSJsLAGtn",4,"14-128-190-5944",2866.83,"MACHINERY"," requests. final, regular ideas sleep final accou").\ncustomer(5,"Customer#000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794.47,"HOUSEHOLD","n accounts will have to unwind. foxes cajole accor",black,"-",5,"Customer#000000005","KvpyuHCplrB84WgAiGV6sYpZq7Tj",3,"13-750-942-6364",794.47,"HOUSEHOLD","n accounts will have to unwind. foxes cajole accor").\ncustomer(6,"Customer#000000006","sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn",20,"30-114-968-4951",7638.57,"AUTOMOBILE","tions. even deposits boost according to the slyly bold packages. final accounts cajole requests. furious",black,"-",6,"Customer#000000006","sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn",20,"30-114-968-4951",7638.57,"AUTOMOBILE","tions. even deposits boost according to the slyly bold packages. final accounts cajole requests. furious").\ncustomer(7,"Customer#000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561.95,"AUTOMOBILE","ainst the ironic, express theodolites. express, even pinto beans among the exp",black,"-",7,"Customer#000000007","TcGe5gaZNgVePxU5kRrvXBfkasDTea",18,"28-190-982-9759",9561.95,"AUTOMOBILE","ainst the ironic, express theodolites. express, even pinto beans among the exp").\ncustomer(8,"Customer#000000008","I0B10bB0AymmC, 0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819.74,"BUILDING","among the slyly regular theodolites kindle blithely courts. carefully even theodolites haggle slyly along the ide",black,"-",8,"Customer#000000008","I0B10bB0AymmC, 0PrRYBCP1yGJ8xcBPmWhl5",17,"27-147-574-9335",6819.74,"BUILDING","among the slyly regular theodolites kindle blithely courts. carefully even theodolites haggle slyly along the ide").\ncustomer(9,"Customer#000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324.07,"FURNITURE","r theodolites according to the requests wake thinly excuses: pending requests haggle furiousl",black,"-",9,"Customer#000000009","xKiAFTjUsCuxfeleNqefumTrjS",8,"18-338-906-3675",8324.07,"FURNITURE","r theodolites according to the requests wake thinly excuses: pending requests haggle furiousl").\ncustomer(10,"Customer#000000010","6LrEaV6KR6PLVcgl2ArL Q3rqzLzcT1 v2",5,"15-741-346-9870",2753.54,"HOUSEHOLD","es regular deposits haggle. fur",black,"-",10,"Customer#000000010","6LrEaV6KR6PLVcgl2ArL Q3rqzLzcT1 v2",5,"15-741-346-9870",2753.54,"HOUSEHOLD","es regular deposits haggle. fur").\ncustomer(11,"Customer#000000011","PkWS 3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING","ckages. requests sleep slyly. quickly even pinto beans promise above the slyly regular pinto beans. ",black,"-",11,"Customer#000000011","PkWS 3HlXqwTuzrKg633BEi",23,"33-464-151-3439","-272.60","BUILDING","ckages. requests sleep slyly. quickly even pinto beans promise above t

In [49]:
clingoRules += '\n'+'\n'.join(originalSetup)

In [50]:
ASPRules(clingoRules)

'% schema customer(c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsement,c_comment)\n% schema lineitem(l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment)\n% schema orders(o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment)\n% schema part(p_partkey,p_name,p_mfgr,p_brand,p_type,p_size,p_container,p_rentalprice,p_comment)\n% schema partsupp(ps_partkey,ps_suppkey,ps_availqty,ps_supplycost,ps_comment)\n% schema supplier(s_suppkey,s_name,s_address,s_nationkey,s_phone,s_acctbal,s_comment)\n\ncustomer(1,Customer#000000001,IVhzIApeRb ot,c,E,15,25-989-741-2988,711.56,BUILDING,to the even, regular platelets. regular, ironic epitaphs nag e,black,"-",1,Customer#000000001,IVhzIApeRb ot,c,E,15,25-989-741-2988,711.56,BUILDING,to the even, regular platelets. regular, ironic epitaphs nag e).\ncustomer(2,Customer#000000002,XSTf4,NCwDVaWNe6tEgvwfmRchLXak,13,23-768-687-3665,121.65,AUTOMOBILE,l accounts. blithely ironic theodolites integrate boldly: caref,black,"-",2,Customer#000000002,XSTf4,NCwDVaWNe6tEgvwfmRchLXak,13,23-768-687-3665,121.65,AUTOMOBILE,l accounts. blithely ironic theodolites integrate boldly: caref).\ncustomer(3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7498.12,AUTOMOBILE, deposits eat slyly ironic, even instructions. express foxes detect slyly. blithely even accounts abov,black,"-",3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7498.12,AUTOMOBILE, deposits eat slyly ironic, even instructions. express foxes detect slyly. blithely even accounts abov).\ncustomer(4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2866.83,MACHINERY, requests. final, regular ideas sleep final accou,black,"-",4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2866.83,MACHINERY, requests. final, regular ideas sleep final accou).\ncustomer(5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,794.47,HOUSEHOLD,n accounts will have to unwind. foxes cajole accor,black,"-",5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,794.47,HOUSEHOLD,n accounts will have to unwind. foxes cajole accor).\ncustomer(6,Customer#000000006,sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn,20,30-114-968-4951,7638.57,AUTOMOBILE,tions. even deposits boost according to the slyly bold packages. final accounts cajole requests. furious,black,"-",6,Customer#000000006,sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn,20,30-114-968-4951,7638.57,AUTOMOBILE,tions. even deposits boost according to the slyly bold packages. final accounts cajole requests. furious).\ncustomer(7,Customer#000000007,TcGe5gaZNgVePxU5kRrvXBfkasDTea,18,28-190-982-9759,9561.95,AUTOMOBILE,ainst the ironic, express theodolites. express, even pinto beans among the exp,black,"-",7,Customer#000000007,TcGe5gaZNgVePxU5kRrvXBfkasDTea,18,28-190-982-9759,9561.95,AUTOMOBILE,ainst the ironic, express theodolites. express, even pinto beans among the exp).\ncustomer(8,Customer#000000008,I0B10bB0AymmC, 0PrRYBCP1yGJ8xcBPmWhl5,17,27-147-574-9335,6819.74,BUILDING,among the slyly regular theodolites kindle blithely courts. carefully even theodolites haggle slyly along the ide,black,"-",8,Customer#000000008,I0B10bB0AymmC, 0PrRYBCP1yGJ8xcBPmWhl5,17,27-147-574-9335,6819.74,BUILDING,among the slyly regular theodolites kindle blithely courts. carefully even theodolites haggle slyly along the ide).\ncustomer(9,Customer#000000009,xKiAFTjUsCuxfeleNqefumTrjS,8,18-338-906-3675,8324.07,FURNITURE,r theodolites according to the requests wake thinly excuses: pending requests haggle furiousl,black,"-",9,Customer#000000009,xKiAFTjUsCuxfeleNqefumTrjS,8,18-338-906-3675,8324.07,FURNITURE,r theodolites according to the requests wake thinly excuses: pending requests haggle furiousl).\ncustomer(10,Customer#000000010,6LrEaV6KR6PLVcgl2ArL Q3rqzLzcT1 v2,5,15-741-346-9870,2753.54,HOUSEHOLD,es regular deposits haggle. fur,black,"-",10,Cu

In [ ]:
sqlCode = 'lineitem("-","-","-","-",Q,EP,D,"-",RF,LS,SD,"-","-","-","-","-") :- lineitem(_,_,_,_,Q,EP,D,_,RF,LS,SD,_,_,_,_,_),SD<'

In [ ]:
tableLayoutLineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice',
                       'l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate',
                      'l_receiptdate','l_shipinstruct','l_shipmode','l_comment']

In [81]:
t = Time("1996-01-18", format='isot', scale='utc')